## **Tnum** utility class for accessing Truenumbers&reg;


**Tnum** is a Python utility class of functions for accessing the Truenumbers (TN) REST API to create, query and work with truenumber data.  It includes utilities for ingesting **pandas** dataframes into TN. This notbook runs though these functions with examples as a kind of introduction and user guide for **Tnum** and TN. 

We start by importing the class, and calling *authorize* to connect to a TN server and display all the available numberspaces in that server.  We then set **Tnum** state to work with one particular numberspace.


In [1]:
from IPython.display import Markdown, display
from tnum import Tnum
import pandas as pd

Tnum.authorize("gov.truenumbers.com")
Tnum.createSpace("scratchspace02")
Tnum.setSpace("scratchspace02")
Tnum.delete_by_query("* has *")

Available spaces: ['test-nard-parser', 'tyler-test', 'TET-DEMO-SANDBOX1', 'scratchspace01', 'scratchspace02', 'scratch01']
Defaulting to: test-nard-parser
Creating scratchspace02 NUMBERSPACE_ALREADY_EXISTS
Numberspace set to: scratchspace02
Deleted 0  matches


### Ingesting data into TN
Next, we open a CSV file in **pandas** to get a data frame of solar system information

In [2]:
df = pd.read_csv ("C:/Users/allen/Documents/Jupyter/sol_data.csv")
pd.set_option('display.max_columns', None)
df.head()

,eName,isPlanet,semimajorAxis,perihelion,aphelion,eccentricity,inclination,density,gravity,escape,meanRadius,equaRadius,polarRadius,flattening,dimension,sideralOrbit,sideralRotation,discoveryDate,mass_kg,volume,orbit_type,orbits,bondAlbido,geomAlbido,RV_abs,p_transit,transit_visibility,transit_depth,massj,semimajorAxis_AU,grav_int
0,Moon,False,384400.0,363300,405500,0.0549,5.145,3.344,1.6200,2380.000,33.0,1738.1,1736.0,0.0012,NaN,27.32170,655.72800,NaN,7.350000e+22,2.196800e+10,Secondary,Earth,NaN,NaN,NaN,1.811589,326.086108,2.250000e-09,3.870000e-05,0.002570,6.610000e+25
1,Phobos,False,9376.0,9234,9518,0.0151,1.075,1.900,0.0057,11.390,33.0,13.0,9.1,0.0000,26.8 x 22.4 x 18.4,0.31891,0.76530,12/08/1877,1.060000e+16,5.783610e+03,Secondary,Mars,NaN,NaN,NaN,74.272078,13368.973980,2.250000e-09,5.580000e-12,0.000063,1.600000e+22
2,Deimos,False,23458.0,23456,23471,0.0002,1.075,1.750,0.0030,5.556,33.0,7.8,5.1,0.0000,15.0 x 12 x 10.4,1.26244,30.29856,12/08/1877,2.400000e+15,9.997800e+03,Secondary,Mars,NaN,NaN,NaN,29.686035,5343.486231,2.250000e-09,1.260000e-12,0.000157,5.790000e+20
3,Io,False,421800.0,0,0,0.0040,0.036,3.530,1.7900,0.000,1821.5,0.0,0.0,0.0000,NaN,1.76914,42.45930,07/01/1610,8.930000e+22,NaN,Secondary,Jupiter,NaN,NaN,NaN,1.655200,297.936060,6.840000e-06,4.700000e-05,0.002820,6.670000e+25
4,Europa,False,671100.0,0,0,0.0090,0.466,3.010,1.3100,0.000,1560.8,0.0,0.0,0.0000,NaN,3.55118,85.22930,08/01/1610,4.800000e+22,NaN,Secondary,Jupiter,NaN,NaN,NaN,1.039939,187.188950,5.020000e-06,2.530000e-05,0.004486,1.420000e+25


This dataset was sourced from [kaggle.com](https://www.kaggle.com/datasets/jaredsavage/solar-system-major-bodies-data). The source reports that some of the data was retrieved from research papers and some of it was calculated.  We include descriptions of a few of the 31 columns in the dataset.  

|column name|type|description|
|:--|:--|:--|
|eName|string|the name of the object|
|isPlanet|boolean|is the object a planet (this includes the five dwarf planets)|
|semimajorAxis|integer|mean orbital distance in km|
|perihelion|integer|closest distance from the Sun during its orbit|
|aphelion|integer|furthest distance from the Sun during its orbit|
|discoveryDate|date|date of discovery, this is left blank for any objects that were known since antiquity|
|mass_kg|integer|total estimated mass of object in kg|
|volume|integer|approximate volume in km^3
|orbit_type|class|either primary; orbites the Sun, or secondary; orbits a planet|
|orbits|class|the planet that the body orbits. If it does not orbit a planet then it is NA|
|semimajorAxis_AU|integer|orbital radius in Astronomical Units|
|grav_int|double|gravitational interaction with the Sun in Newtons|

Each row in the frame gives information about a particular body in our solar system, primarily identified by name in the first column. It's *orbit_type* is primary or secondary if it orbits the sun or something else.  But if it's primary, the *orbits* column contains NA instead of "sun".  The *isPlanet* column (note that some column names use underscores and other camel-case) identifies official planets but there is no indication of comets, minor planets, dwarf planets or other categories of objects, though some names like **Halley's Comet** give an indication.  

For TN we want a more conversational identification like, say, **Phobos orbiter of planet Mars** or **Hydra orbiter of dwarf planet Pluto**.  We can build these from a template that references the relevant columns.

      $tokenize(eName) orbiter of $enhance_planet(orbits)

where macro string *$tokenize(eName)*, for example, indicates taking the value in the *eName* column, processing it with a Python function *tokenize* and replacing the macro string with the result.  The *orbited* function is particular to this data set and defined below

In [3]:
def orbited(str):
    token = tokenize(str)
    if token == 'NA':
        token = "Sun"

    match token:
        case "Pluto":
            token = "dwarf planet Pluto"
        case "1_Ceres":
            token = "dwarf planet 1_Ceres"
        case "136199_Eris":
            token = "dwarf planet 136199_Eris"
            
    
        
    
def enhance_planet(str):
    if str == 'TRUE':
        

SyntaxError: incomplete input (3398835597.py, line 19)